# Rice Encoding

# Encoding
1. Fix an integer value K.
K = 4 or 2
<br>
2. Compute the modulus, M by using the equation  𝑀=2**K
<br>
3. For S, the number to be encoded, find<br>
A. quotient =  𝑞=𝑖𝑛𝑡(𝑆/𝑀)<br>
B. remainder =  𝑟=𝑆𝑚𝑜𝑑𝑢𝑙𝑜𝑀
<br>
4. Generate Codeword<br>
A. The Quotient_Code is q in unary format.<br>
B. The Remainder_Code is r in binary using K bits.<br>
C. The Codeword will have the format <Quotient_Code> <Remainder_Code>

# Decoding
1. Determine q by counting the number of 1s before the first 0.<br>
2. Determine r reading the next K bits as a binary value.<br>
3. Write out S, the encoded number, as q × M + r.<br>

In [1]:
import numpy as np
import math
import sounddevice as sd
import os
from scipy.io import wavfile

In [2]:
def diy_rice_encoder(S, k):
    # If sample value is negative, convert to positive
    ## Store actual sign separately
    if S < 0:
        S = -S
        sign = "-"
    else:
        sign = "+"

    # Compute the modulus, M by using the equation  𝑀=2**k
    M = 2**k

    # For S, the number to be encoded, find
    ## quotient =  𝑞 = 𝑖𝑛𝑡(𝑆/𝑀)
    quotient = int(S//M)
    ## remainder =  𝑟 = 𝑆𝑚𝑜𝑑𝑢𝑙𝑜𝑀
    remainder = S%M

    # Generate Codeword
    quotient_code = "1"*quotient + "0"


    # Write out r in binary
    remainder_code = format(remainder, "0{}b".format(k))

    # Encoded string
    codeword = sign + quotient_code + "0" + str(remainder_code)

    return codeword

In [3]:
def diy_rice_decoder(S, k):
    # Remove the sign and store separately
    sign = S[0]
    S = S[1:]

    # Compute the modulus, M by using the equation  𝑀=2**k
    M = 2**k

    # Separate quotient and remainder into a list of two items by spliting at the first 0
    codeword = S.split("0", 1)

    # Determine q by counting the number of 1s before the first 0 
    ## i.e. the length of the string in the first item of the list
    q = len(codeword[0])

    # Determine r reading the next K bits as a binary value.
    ## i.e. take K bits from the second item and convert into decimal value
    r = int(codeword[1][:k], 2)

    # Write out S, the encoded number, as q × M + r.
    S = q*M + r

    # Change sign of the decoded bit back to actual sign
    if sign == "-":
        S = -S

    return S

In [4]:
def diy_audio_compress(filename,k):
    # Read the audio file
    sample_rate, audio = wavfile.read(filename)

    # Play original audio
    print("Playing audio from "+filename)
    sd.play(audio, sample_rate)

    # Encode the audio file and save as <filename>_Enc.ex2.ex2
    encoded_filename = filename.split(".")[0]+"_Enc.ex2"

    with open(encoded_filename, "wb") as encodedfile:
        for i in range(len(audio)):
            encoded_data = diy_rice_encoder(audio[i], k) + "\n"
            #
            encodedfile.write(encoded_data.encode())
    encodedfile.close()
    print("Audio successfully coded into file: "+encoded_filename)
    print("Sample Rate: "+str(sample_rate))

In [5]:
def diy_audio_decompress(encoded_filename,k,sample_rate):
    # Decode the encoded audio file
    decoded_audio = []
    with open(encoded_filename, "rb") as encodedfile:
        for line in encodedfile:
            decoded_audio.append(diy_rice_decoder(line.decode("utf8").strip(), k))
    encodedfile.close()
    decoded_audio = np.array(decoded_audio, dtype="int16")

    # Play decoded audio data
    print("Playing audio decoded from "+encoded_filename)
    sd.play(decoded_audio, sample_rate)

    # Save decoded audio data into .wav format
    decoded_filename = encoded_filename.split(".")[0]+"Dec.wav"
    wavfile.write(decoded_filename, sample_rate, decoded_audio)

# Compression and decompression with k = 4 bits

In [6]:
file1 = "Sound1.wav"
f1_size = math.ceil(os.path.getsize(file1)/1024)
print("File Size is :", f1_size, "kilobytes")
diy_audio_compress(file1,4)

File Size is : 979 kilobytes
Playing audio from Sound1.wav
Audio successfully coded into file: Sound1_Enc.ex2
Sample Rate: 44100


In [7]:
file2 = "Sound2.wav"
f2_size = math.ceil(os.path.getsize(file2)/1024)
print("File Size is :", f2_size, "kilobytes")
diy_audio_compress(file2,4)

File Size is : 985 kilobytes
Playing audio from Sound2.wav
Audio successfully coded into file: Sound2_Enc.ex2
Sample Rate: 44100


In [8]:
encfile1 = "Sound1_Enc.ex2"
ef1_size = math.ceil(os.path.getsize(encfile1)/1024)
print("File Size is :", ef1_size, "kilobytes")
diy_audio_decompress(encfile1,4,44100)

File Size is : 26106 kilobytes
Playing audio decoded from Sound1_Enc.ex2


In [9]:
encfile2 = "Sound2_Enc.ex2"
ef2_size = math.ceil(os.path.getsize(encfile2)/1024)
print("File Size is :", ef2_size, "kilobytes")
diy_audio_decompress(encfile2,4,44100)

File Size is : 154436 kilobytes
Playing audio decoded from Sound2_Enc.ex2


In [10]:
# calculate the percentage compression for file 1
print("Original size of {} is {} kilobytes".format(file1,f1_size))
print("Compressed size of {} is {} kilobytes".format(file1, ef1_size))
percent_origin_1 = round(ef1_size/f1_size*100)
print("This is {} percent of the original file".format(percent_origin_1))

# calculate the percentage compression for file 2
print("Original size of {} is {} kilobytes".format(file2,f2_size))
print("Compressed size of {} is {} kilobytes".format(file2, ef2_size))
percent_origin_2 = round(ef2_size/f2_size*100)
print("This is {} percent of the original file".format(percent_origin_2))

Original size of Sound1.wav is 979 kilobytes
Compressed size of Sound1.wav is 26106 kilobytes
This is 2667 percent of the original file
Original size of Sound2.wav is 985 kilobytes
Compressed size of Sound2.wav is 154436 kilobytes
This is 15679 percent of the original file


# Compression and decompression with k = 2 bits

In [11]:
file1 = "Sound1.wav"
f1_size = math.ceil(os.path.getsize(file1)/1024)
print("File Size is :", f1_size, "kilobytes")
diy_audio_compress(file1,2)

File Size is : 979 kilobytes
Playing audio from Sound1.wav
Audio successfully coded into file: Sound1_Enc.ex2
Sample Rate: 44100


In [12]:
file2 = "Sound2.wav"
f2_size = math.ceil(os.path.getsize(file2)/1024)
print("File Size is :", f2_size, "kilobytes")
diy_audio_compress(file2,2)

File Size is : 985 kilobytes
Playing audio from Sound2.wav
Audio successfully coded into file: Sound2_Enc.ex2
Sample Rate: 44100


In [13]:
encfile1 = "Sound1_Enc.ex2"
ef1_size = math.ceil(os.path.getsize(encfile1)/1024)
print("File Size is :", ef1_size, "kilobytes")
diy_audio_decompress(encfile1,2,44100)

File Size is : 92255 kilobytes
Playing audio decoded from Sound1_Enc.ex2


In [14]:
encfile2 = "Sound2_Enc.ex2"
ef2_size = math.ceil(os.path.getsize(encfile2)/1024)
print("File Size is :", ef2_size, "kilobytes")
diy_audio_decompress(encfile2,2,44100)

File Size is : 605684 kilobytes
Playing audio decoded from Sound2_Enc.ex2


In [15]:
# calculate the percentage compression for file 1
print("Original size of {} is {} kilobytes".format(file1,f1_size))
print("Compressed size of {} is {} kilobytes".format(file1, ef1_size))
percent_origin_1 = round(ef1_size/f1_size*100)
print("This is {} percent of the original file".format(percent_origin_1))

# calculate the percentage compression for file 2
print("Original size of {} is {} kilobytes".format(file2,f2_size))
print("Compressed size of {} is {} kilobytes".format(file2, ef2_size))
percent_origin_2 = round(ef2_size/f2_size*100)
print("This is {} percent of the original file".format(percent_origin_2))

Original size of Sound1.wav is 979 kilobytes
Compressed size of Sound1.wav is 92255 kilobytes
This is 9423 percent of the original file
Original size of Sound2.wav is 985 kilobytes
Compressed size of Sound2.wav is 605684 kilobytes
This is 61491 percent of the original file


|  | Original Size | Rice (K = 4 bits) | Rice (K = 2 bits) | % Compression (K = 4 bits) | % Compression (K = 2 bits) |
| --- | --- | --- | --- | --- | --- |
| Sound1.wav | 979KB | 26106KB | 92255KB | 2667% of original | 9423% of original |
| --- | --- | --- | --- | --- | --- |
| Sound2.wav | 985KB | 154436KB | 605684KB | 15679% of original | 61491% of original |

## Further development: Optimal K

In [16]:
def optimal_k_generator(filename):
    # Read audio file
    sample_rate, audio = wavfile.read(filename)

    # Find the maximum absolute value of audio samples
    max_value = max(abs(audio))
    
    # initialise two lists to store values
    Qs = []
    Rs = []
    Ds = []

    # Find the optimal k value
    for i in range(1,17):
        M = 2**i
        R = max_value%M
        Q = (max_value-R)/M
        Qs.append(Q)
        Rs.append(R)
    for j in range(len(Qs)):
        D = math.sqrt(((0-Qs[j])**2 + (0-Rs[j])**2))
        Ds.append(D)
    # add one to restore to a k value
    best_case = Ds.index(min(Ds))+1
    return best_case

In [19]:
k1 = optimal_k_generator(file1)
print(k1)
k2 = optimal_k_generator(file2)
print(k2)

7
7


In [21]:
f1_size = math.ceil(os.path.getsize(file1)/1024)
print("File Size is :", f1_size, "kilobytes")
diy_audio_compress(file1,k1)

File Size is : 979 kilobytes
Playing audio from Sound1.wav
Audio successfully coded into file: Sound1_Enc.ex2
Sample Rate: 44100


In [22]:
file2 = "Sound2.wav"
f2_size = math.ceil(os.path.getsize(file2)/1024)
print("File Size is :", f2_size, "kilobytes")
diy_audio_compress(file2,k2)

File Size is : 985 kilobytes
Playing audio from Sound2.wav
Audio successfully coded into file: Sound2_Enc.ex2
Sample Rate: 44100


In [23]:
encfile1 = "Sound1_Enc.ex2"
ef1_size = math.ceil(os.path.getsize(encfile1)/1024)
print("File Size is :", ef1_size, "kilobytes")
diy_audio_decompress(encfile1,k1,44100)

File Size is : 8037 kilobytes
Playing audio decoded from Sound1_Enc.ex2


In [24]:
encfile2 = "Sound2_Enc.ex2"
ef2_size = math.ceil(os.path.getsize(encfile2)/1024)
print("File Size is :", ef2_size, "kilobytes")
diy_audio_decompress(encfile2,k2,44100)

File Size is : 24012 kilobytes
Playing audio decoded from Sound2_Enc.ex2


In [25]:
# calculate the percentage compression for file 1
print("Original size of {} is {} kilobytes".format(file1,f1_size))
print("Compressed size of {} is {} kilobytes".format(file1, ef1_size))
percent_origin_1 = round(ef1_size/f1_size*100)
print("This is {} percent of the original file".format(percent_origin_1))

# calculate the percentage compression for file 2
print("Original size of {} is {} kilobytes".format(file2,f2_size))
print("Compressed size of {} is {} kilobytes".format(file2, ef2_size))
percent_origin_2 = round(ef2_size/f2_size*100)
print("This is {} percent of the original file".format(percent_origin_2))

Original size of Sound1.wav is 979 kilobytes
Compressed size of Sound1.wav is 8037 kilobytes
This is 821 percent of the original file
Original size of Sound2.wav is 985 kilobytes
Compressed size of Sound2.wav is 24012 kilobytes
This is 2438 percent of the original file
